# 8.4: Inspecting `HCLG.fst`

In the previous notebooks we ran `run_compile_graph.sh` which built a large `FST` called `HCLG.fst`.  And since we ran this command for **each** of the three acoustic models we built, you will see an `HCLG.fst` in each of the three "main" subdirectories of `exp`: `monophones`, `triphones`, and `triphones_lda`.

In [ ]:
%%bash
ls -lah exp/monophones/graph/HCLG.fst

In [ ]:
%%bash
ls -lah exp/triphones/graph/HCLG.fst

In [ ]:
%%bash
ls -lah exp/triphones_lda/graph/HCLG.fst

The first thing you should notice is that, perhaps not surprisingly, the `HCLG` is larger for each of the subsequent levels of the acoustic model.

Before we inspect the `HCLG` in more detail, take the time to read [this **excellent** blog post](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html) (from one of the main contributors of `kaldi`) that describes in detail the makeup of the `HCLG`.  

You'll also want to read **Section 6** of [this tutorial](https://github.com/michaelcapizzi/kaldi/blob/kaldi_instructional/egs/INSTRUCTIONAL/resource_files/resources/wfst_tutorial.pdf) which is a summary of the [original paper](https://github.com/michaelcapizzi/kaldi/blob/kaldi_instructional/egs/INSTRUCTIONAL/resource_files/resources/wfst_paper.pdf) introducing the idea of an `HCLG`.

## Building `HCLG`

The `HCLG` is a single `FST` that represents **all** aspects of our `ASR` pipeline into one graph.  As you read, it is the `composition` of **four** separate `FST`s (Note that they are `composed` in reverse order.

Decoding (when we take a new audio file and predict what was said) comes down to two steps:

   1. determining which `GMM` best matches the incoming frames
   2. looking for the most likely path through the graph

We can use the same packages we used last week to take a closer look at `HCLG` and its composites.

In [ ]:
# because of the way `kaldi` installed `openFST` we have to add the path to the python functions here
import sys
sys.path.append("/scratch/kaldi/tools/openfst-1.6.2/lib/python2.7/site-packages")    

from utils.fst_manipulate import fst_manipulate as fstman  # scripts to further manipulate fsts

import pywrapfst as openfst  # the wrapper module
import graphviz as dot       # a wrapper for graphviz, which will allow us to visualize

### $G$

`G` is the `FST` representation of our `language model`.  We looked at this in detail in the last week's notebooks.  It has `word:word` on its edges.  Below is our `language model` built from the animal corpus.

In [ ]:
G_animal = openfst.Fst.read("resource_files/fst/animal_fst-2_gram.fst")
G_animal

The "real" `G.fst` can be found in `data/lang_test_tg` and was built way back in week 3 when we ran `run_prepare_data.sh` to build the `data` directory.

**Note:** These composite `FST`s will be too big to visualize, but we can still gather some information about them.

In [ ]:
%%bash
ls -lah data/lang_test_tg/G.fst

In [ ]:
G = openfst.Fst.read("data/lang_test_tg/G.fst")

We can calculate the number of states by converting the `iterator` in `.states()` to a `<list>` and getting its length.

In [ ]:
G_states = len(list(G.states()))
G_states

We can do a similar thing with the `.arcs()` `iterator` with a small modification.  `.arcs()` takes one argument, a `state`, and so we loop through all `state`s, and then count how many `arc`s that state has.

In [ ]:
G_arcs = 0
for s in G.states():
    intermediate_arcs = len(list(G.arcs(s)))
    G_arcs += intermediate_arcs
G_arcs

Below are some quick functions to do this:

In [ ]:
def get_num_states(fst_in):
    return len(list(fst_in.states()))

def get_num_arcs(fst_in):
    num_arcs = 0
    for s in fst_in.states():
        intermediate_arcs = len(list(fst_in.arcs(s)))
        num_arcs += intermediate_arcs
    return num_arcs

We can also get some information about the `arc`s.

**Note:** We'll just look at the first **five** states and then `break` out of our nested `for` loops.

In [ ]:
c = 0
for s in G.states():
    for a in G.arcs(s):
        if c < 5:
            print("in: {}\nout: {}\nweight: {}".format(
                a.ilabel,
                a.olabel,
                a.weight
                )
            )
            print("=========")
        else:
            break
        c += 1
    break

Remember that `openfst` will use `index`es instead of `string`s on the `arc`s, but we can recover the words by looking at the "lookup" file `kaldi` built for this very purpose.

In [ ]:
%%bash
cat data/lang/words.txt | grep -E " 2$"       # the -E flag will allow us to use a regex
cat data/lang/words.txt | grep -E " 49$"
cat data/lang/words.txt | grep -E " 220$"
cat data/lang/words.txt | grep -E " 227$"
cat data/lang/words.txt | grep -E " 456$"

You'll notice that the `ilabel` and `olabel` are the same, confirming that the `arc`s consist of `word:word`.  And the `weight` is the representation of the probability (see `7.2 Examining G_fst`).

### $L$
`L` is the `FST` representation of our `lexicon`.  This was built in week 3 when we built the `data` directory (see `3.2: Inspecting data dir`).  

In [ ]:
ls -lah data/lang/ | grep L

In [ ]:
L = openfst.Fst.read("data/lang/L.fst")
print("number of states in L: {}".format(get_num_states(L)))
print("number of arcs in L: {}".format(get_num_arcs(L)))

In [ ]:
c = 0
for s in L.states():
    for a in L.arcs(s):
        if c < 15:
            print("in: {}\nout: {}\nweight: {}".format(
                a.ilabel,
                a.olabel,
                a.weight
                )
            )
            print("=========")
        else:
            break
        c += 1
    if c > 15:
        break

`L` has a different `arc` structure, consisting of `phone:word`.  And so to understand these `arc`s we need to also access the `phones` "lookup".

In [ ]:
%%bash
cat data/lang/phones.txt | grep -E " 122$"
cat data/lang/words.txt | grep -E " 8$"

It's not clear what the `weight` represents, but you'll notice that it's almost always **the same** value (or 0).  So if the weight is equal for all `arc`s, it is essentially unimportant in determining shortest path.

**Note:** You may also be wondering how this will work for words consisting of more than one phone.  The **first** `arc` will have `phone:word` and each subsequent arc will have `phone:<eps>` (`<eps>` representing $epsilon$, or the empty `arc`).  You can see an example of this in the visualization of $L$ in this [blog post](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html).

### $LG = L \circ G$

When we `compose` `L` and `G`, we will get a representation that incorporates **both** spelling **and** language model information.  We will end up with an `FST` that has the `arc` structure of `phone:word`.  

**Note:** In general, a `composition` will keep the arc **`input`** of the first `FST` and the arc **`output`** of the second `FST`.

If you did **NOT** delete the `tmp` folder in `data/lang_test_tg`, then you'll have a copy of an `LG.fst` built from the last `run_compile_graph.sh` command you ran.

In [ ]:
%%bash
ls -lah data/lang_test_tg/tmp/LG.fst

It shouldn't be surprising that it is **much** larger than `L` and `G`.

In [ ]:
LG = openfst.Fst.read("data/lang_test_tg/tmp/LG.fst")

In [ ]:
c = 0
for s in LG.states():
    for a in LG.arcs(s):
        if c < 10:
            print("in: {}\nout: {}\nweight: {}".format(
                a.ilabel,
                a.olabel,
                a.weight
                )
            )
            print("=========")
        else:
            break
        c += 1
    if c > 10:
        break

In [ ]:
%%bash
cat data/lang/phones.txt | grep -E " 14$"
cat data/lang/words.txt | grep -E " 79970$"

And remember that the `arc` `weights` from $G$ represented the language model probabilities, so these are propogated to $LG$.

### $C$

`C` is a representation of the `triphone` information from our acoustic model. 

**Note:** This `FST` isn't explicitly built in `kaldi` as a standalone item, and so we can't examine it like we did `G` and `L`.  But [this blog post](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html) has an image of what it looks like.  It also explains the indexing in a very straightforward manner:

 ```
 The input symbols of the C graph are triphone IDs, which are specified by using a Kaldi-specific data structure called ilabel_info(frankly clabel_info would have been more intuitive name for me, but perhaps there is reason it's called that way). Basically this is an array of arrays, where the the indices of the first dimension are the triphone IDs and the individual entries of the nested arrays are the IDs of the context-independent phones, which constitute the context window for the particular triphone. For example if there are triphone "a/b/c"(i.e. central phone "b" with left context "a" and right context "c") with id "10" the eleventh entry in the ilabel_info will be an array containing the context-independent ID's of the phones "a", "b" and "c".```
 
So to simplify, the `arc` structure is `triphone:phone` where `phone` in this case refers to the `ARPAbet` `phones` we are using to represents our words.

In [ ]:
%%bash
head data/lang/phones.txt

### $CLG = C \circ LG$

Since this is a composition of `C` (`arc` structure of `triphone:phone`) and `LG` (`arc` structure of `phone:word`), the resulting `FST` will have `arc` structure of `triphone:word`.  And the `arc` `weight`s are again a propogation of our language model probabilities.

### $H$

Remember that our acoustic model is ultimately made up of an `HMM`, and so the `H` here incorporates that information into our graph.  Its `arc` structure is `transition_id:triphone` where `transition_id` consists of four parts:
 1. a `phone`
 2. an `HMM` `state`
 3. a `PDF`
 4. an `HMM` outgoing `arc`
 
 
[This blog post](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html) gives an example:

```
For example "k_1_739_1" in this notation means that this is the transition-id associated with the state "1"(i.e. the second state) of the monophone "k" having PDF ID of 739(this is in general different for the different context-dependent versions of "k") and the outgoing arc from this HMM state, which has ID "1". 
```

### $HCLG = H \circ CLG$

Our final graph is a composition of all four of the individual parts above.  And most importantly it has an `arc` structure of `transition_id:word`.  And this is **exactly** what we need in order to make predictions on new audio.  For any given frame, our `GMM` will make a prediction as to which `transition_id` to assign it, and given that information for each consecutive frame, we can work our way through the graph to make a prediction as to what `word`s are represented by the sequence of frames.

**Note:** You'll actually see **both** an `HaCLG.fst` and an `HCLG.fst`.  The `HaCLG.fst` is simply an intermediate form (if you're really interested as to why it's required, it's explained briefly [here](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html)).  We are only going to pay attention to the **final** version: `HCLG.fst`.

## Comparing `HCLG` from different acoustic models

We have built **three** different `HCLG.fst`s:
 1. one built from our `monophone` acoustic model
 2. one built from our `triphone` acoustic model
 3. one built from our `triphone_lda` acoustic model

But they are different sizes:

In [ ]:
%%bash
ls -lah exp/monophones/graph/HCLG.fst
ls -lah exp/triphones/graph/HCLG.fst
ls -lah exp/triphones_lda/graph/HCLG.fst

In [ ]:
HCLG_mono = openfst.Fst.read("exp/monophones/graph/HCLG.fst")
HCLG_tri = openfst.Fst.read("exp/triphones/graph/HCLG.fst")
HCLG_tri_lda = openfst.Fst.read("exp/triphones_lda/graph/HCLG.fst")

print("number of states compared\nmono: {}\ntri: {}\ntri_lda: {}\n\n".format(
        get_num_states(HCLG_mono),
        get_num_states(HCLG_tri),
        get_num_states(HCLG_tri_lda)
    )
)
print("====================")
print("number of arcs compared\nmono: {}\ntri: {}\ntri_lda: {}".format(
        get_num_arcs(HCLG_mono),
        get_num_arcs(HCLG_tri),
        get_num_arcs(HCLG_tri_lda)
    )
)

It **should** be clear that we used the **exact same** `L` and `G` for all three of these, and so the only things that could have **possibly** changed were the `C` and `H`, which makes sense since **both** of these individual `FST`s are built from parts of the acoustic model.

You may remember from `6.4: Examining the acoustic models` that the `triphone` model actually had a **smaller** `decision tree` than the `monophone` model because it did some state-tying (*e.g.* clustering).  And you may also remember that the `triphone_lda` model had a **larger** `decision tree` than the `triphone` model because we increased the number of leaves that we would allow.  The number of leaves of the acoustic model `decision tree`s directly impact the size of `H` which explains the differences in the number of states for each `HCLG` above. 

These size differences will have an impact of the **time** it takes to decode a new audio sample...something we'll look at more in the next two weeks.